In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
import torch

In [6]:

documents = SimpleDirectoryReader("Policies/").load_data()

In [7]:

len(documents)

43

In [8]:

documents[10]

Document(id_='20333bb2-dc6c-478f-949f-719f1396f109', embedding=None, metadata={'page_label': '11', 'file_name': 'EmployeeHandbook.pdf', 'file_path': 'c:\\Clicflyer\\LangChain\\TinyLLama\\Policies\\EmployeeHandbook.pdf', 'file_type': 'application/pdf', 'file_size': 179561, 'creation_date': '2024-03-22', 'last_modified_date': '2024-03-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=' \nReceiving a bribe: A supplier gives your nephew a job, but makes it clear that in return they \nexpect you to use your influence in our organization to ensure we continue to do business \nwith them. It is an offence for a supplier to make such an offer. It would be an offence fo r  \nyou to accept the offer as you would be doing so to gain a personal  advantage.  \n \nH

In [9]:
from langchain_core.prompts import PromptTemplate


prompt_template = """
    <|USER|>{query_str}<|ASSISTANT|>
    """


In [10]:
system_prompt = "You are a QA bot. Given the questions answer it correctly."

llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=prompt_template,
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="cpu",
    model_kwargs={"torch_dtype": torch.bfloat16},
)

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

service_context = ServiceContext.from_defaults(
    chunk_size=400,
    llm=llm,
    embed_model=embed_model
)

C:\Users\Shikhar\AppData\Local\Temp\ipykernel_15784\2943767800.py:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [8]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

def predict(input,history):
  response = query_engine.query(input)
  return str(response)


In [9]:
result=predict("What is the number of leaves employee can avail ?","")

: 

In [ ]:
print(result)

21 in a year


In [ ]:
import gradio as gr
gr.ChatInterface(predict).launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://87e49d5dc7fc23fd7c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
